In [1]:
from experiment import ResultsViewer
from IPython.display import Image
import torch
from scipy.io import savemat, loadmat
import os
import json

In [3]:
results = ResultsViewer("experiments/se1_fc")

def lookup(results, *filters):
  r = []
  for res in results:
    skip = False
    for f in filters:
      if not f(res):
        skip = True
        break
    if not skip:
      r.append(res.copy())
  return r



In [4]:
t1l1 = lookup(results, 
            lambda foo: foo['model']['regularization_method'] == 1 and foo['model']['layers'] == 1,
             lambda foo: os.path.exists(""))

In [6]:
len(t1l1)

12

In [12]:
results[0]['matlab_path']

'experiments/server_e1/D=4_R=1296_L=1296_Term=1_Layers=1_Lam=0.001.mat'